In [17]:
# MLflow imports and setup
import mlflow
import mlflow.keras
from mlflow.tracking import MlflowClient

# Configure MLflow to use remote tracking server
mlflow.set_tracking_uri("http://127.0.0.1:8768")
mlflow.autolog()  # Enable autologging for all supported libraries

# Original imports
import keras
import vtk
from keras.models import Model
from keras.layers import (
    Input, 
    concatenate, 
    Conv2D, 
    MaxPooling2D, 
    Conv2DTranspose,
    ZeroPadding2D
)
from keras import backend as K

2025/01/14 15:04:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2025/01/14 15:04:52 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.6.0, but the installed version is 3.8.0. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.
2025/01/14 15:04:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/01/14 15:04:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [18]:
# vtm dataset maker
from vtm_data import VTK_data

In [19]:
# numpy and matplot lib
import numpy as np
import matplotlib.pyplot as plt

In [20]:
# Start MLflow run and log parameters
with mlflow.start_run(run_name="CNN_Training") as run:
    # training params
    batch_size = 32
    epochs = 100 # number of times through training set
    
    # Log training parameters
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)

🏃 View run CNN_Training at: http://127.0.0.1:8768/#/experiments/1203/runs/a0f97a543c4544739613b02bcfaf5a87
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1203


In [21]:
# VTK_data class implementation
# [Previous VTK_data class code remains the same]

# Create instance and load data
base_directory = "../data"
dataset = VTK_data(base_directory)

# Load the data
geometries, steady_flows = dataset.load_data()

if len(dataset.geometries) > 0 and len(dataset.steady_flows) > 0:
    # Get train and test split
    train_geometries = dataset.geometries[0:dataset.split_line]
    train_steady_flows = dataset.steady_flows[0:dataset.split_line]
    test_geometries = dataset.geometries[dataset.split_line:-1]
    test_steady_flows = dataset.steady_flows[dataset.split_line:-1]
    
    # Log dataset metrics
    mlflow.log_metric("train_samples", len(train_geometries))
    mlflow.log_metric("test_samples", len(test_geometries))
    
    # Convert to numpy arrays
    train_geometries = np.stack(train_geometries, axis=0)
    train_steady_flows = np.stack(train_steady_flows, axis=0)
    test_geometries = np.stack(test_geometries, axis=0)
    test_steady_flows = np.stack(test_steady_flows, axis=0)
    
    # Log data shapes
    mlflow.log_param("input_shape", str(train_geometries.shape[1:]))
    mlflow.log_param("output_shape", str(train_steady_flows.shape[1:]))

AttributeError: 'ImageData' object has no attribute 'Update'

In [22]:
# Model architecture cells combined and wrapped in MLflow
with mlflow.start_run(run_name="Model_Architecture", nested=True):
    # Input layer
    inputs = Input(shape=(9812,))
    
    # Reshape layer
    height, width = find_factors(9812)
    reshaped = Reshape((height, width, 1))(inputs)
    
    # Encoder path
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(reshaped)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Bridge
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    
    # Decoder path
    up6 = concatenate([
        ZeroPadding2D(((1,0),(1,0)))(Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5)), 
        conv4
    ], axis=3)
    
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    
    # Log model architecture
    model = Model(inputs=inputs, outputs=conv6)
    mlflow.keras.log_model(model, "model")
    
    # Log model summary
    model_summary = []
    model.summary(print_fn=lambda x: model_summary.append(x))
    mlflow.log_text("\n".join(model_summary), "model_summary.txt")

🏃 View run Model_Architecture at: http://127.0.0.1:8768/#/experiments/1203/runs/7355d0ae814647c08282540d0a13a8d1
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1203


NameError: name 'find_factors' is not defined